In [1]:
import threading
import serial
import time
import matplotlib.pyplot as plt 
import numpy as np
import pyautogui
from pynput import mouse
import pandas as pd
from bitstring import BitArray
from collections import deque

In [2]:
num_channel = 2

mouse_data = deque()

mouse_times = deque()

muscle_times = deque()

muscle_data = deque()

In [3]:
class Measurer():
    
    
    
    def measure_mouse(self):
        
        while run:
            
            mouse_times.append(time.time())
            
            mouse_data.append(pyautogui.position())
            
#             self.time = time.time()
    
    def measure_muscle(self):
        
        self.ser = serial.Serial('COM4', 230400)
        while run:
            
            muscle_times.append(time.time())

            muscle_data.append(self.ser.read(1))
            
#             self.time = time.time()


In [4]:

measure = Measurer()

In [5]:
run = True
lock = threading.Lock()

t1 = threading.Thread(target=measure.measure_muscle,)
t2 = threading.Thread(target=measure.measure_mouse,)


t1.start(), t2.start()


time.sleep(5)

run = False

t1.join(), t2.join()


(None, None)

In [6]:
muscle_data = [BitArray(val).bin for val in muscle_data]

In [7]:
len(mouse_data), len(muscle_data)

(255584, 33979)

In [8]:
mouse = pd.DataFrame({'mouse_pos':mouse_data, 'mouse_time':mouse_times})
muscle = pd.DataFrame({'muscle_data':muscle_data, 'muscle_time':muscle_times})

In [9]:
count = 0
for val in muscle['muscle_data']:
   
    if int(val[0]):
        break
    else:
        pass
    count += 1
muscle = muscle[count:].reset_index(drop=True)

In [10]:
frames = []
time_ranges = []
for i, val in enumerate(muscle['muscle_data']):
    if bool(int(val[0])):
        frames.append(muscle['muscle_data'][i:i+num_channel*2])
        try:
            time_ranges.append([muscle['muscle_time'][i], muscle['muscle_time'][i + num_channel * 2]])
        except:
            time_ranges.append(None)
    else:
        pass

In [11]:
frames = [list(val) for val in frames]

In [12]:
frames = frames[1:]
time_ranges = time_ranges[1:]

In [13]:
muscle = pd.DataFrame({'muscle_data':frames, 'muscle_time_ranges':time_ranges})

In [14]:
display(muscle.head())
mouse.head()

,muscle_data,muscle_time_ranges
0,"[10000001, 01111001, 00000010, 00110110]","[1583248889.7649817, 1583248889.7649817]"
1,"[10000000, 01001101, 00000010, 00110101]","[1583248889.7649817, 1583248889.7649817]"
2,"[10000000, 01001101, 00000010, 00110100]","[1583248889.7649817, 1583248889.7689776]"
3,"[10000000, 01001101, 00000010, 00110100]","[1583248889.7689776, 1583248889.7689776]"
4,"[10000000, 01001100, 00000010, 00110011]","[1583248889.7689776, 1583248889.7689776]"


,mouse_pos,mouse_time
0,"(869, 829)",1.583249e+09
1,"(869, 829)",1.583249e+09
2,"(869, 829)",1.583249e+09
3,"(869, 829)",1.583249e+09
4,"(869, 829)",1.583249e+09


In [15]:
down_sampled_mouse = []

for val in muscle['muscle_time_ranges']:
    try:
        down_sampled_mouse.append(list(mouse[mouse['mouse_time'] >= val[0]]['mouse_pos'])[0])
    except:
        down_sampled_mouse.append([None, None])


In [16]:
len(down_sampled_mouse)

8490

In [17]:
df = muscle
df['mouse_pos'] = down_sampled_mouse

In [18]:
df['x_pos'] = ([val[0] for val in df['mouse_pos']])
df['y_pos'] = ([val[1] for val in df['mouse_pos']])

    
df.drop(columns = 'mouse_pos', inplace = True)


In [19]:
df.drop(columns='muscle_time_ranges', inplace=True)

df['channel_1'] = [val[:2] for val in df['muscle_data']]
df['channel_2'] = [val[2:] for val in df['muscle_data']]



In [20]:
df.drop(columns='muscle_data', inplace=True)

In [21]:
df.dropna(inplace=True)

In [22]:
def raw_to_num(channel, is_first):
    if is_first:
        
        channel[0] = '0' + channel[0][1:]
  
    channel = channel[0] + channel[1]
    
    channel = int(channel, 2)
    
    return(channel)

In [23]:
df['channel_1'] = [raw_to_num(val, True) for val in df['channel_1']]
df['channel_2'] = [raw_to_num(val, False) for val in df['channel_2']]

In [24]:
df

,x_pos,y_pos,channel_1,channel_2
0,2588.0,593.0,377,566
1,2588.0,593.0,77,565
2,2588.0,593.0,77,564
3,2584.0,583.0,77,564
4,2584.0,583.0,76,563
...,...,...,...,...
8484,143.0,1245.0,34,283
8485,143.0,1245.0,34,283
8486,143.0,1245.0,34,283
8487,143.0,1245.0,34,282


In [25]:
df.to_csv('data/raw_recording.csv', index=False)